# Experiment 8
---
Compare the models SVM , decision tree , random forest and xgboost by tuning their hyperparameters using gridsearchCV

### Steps:
1. Load Dataset
2. Preprocess the data
3. Split Data
4. Define Hyperparameter grid
    - f1_estimators : [50,100,200]
    - max_depth : [None, 10, 20, 30]
    - min_samples_split: [2,5,10]
    - min_samples_leaf: [1,2,4]
5. Perform grid search: Use GridSearchCV with cv=5
6. Train and evaluate: Asess performance on the test set
7. Report results: Print the best parameneters and test accuracy


In [7]:
# Import Libraries
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score

import kagglehub
from kagglehub import KaggleDatasetAdapter


# -----------------------------
# 1. Load Dataset
# -----------------------------

In [8]:
file_path = "heart.csv"
data = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "johnsmith88/heart-disease-dataset",
    file_path,
)
X = data.drop("target", axis=1)
y = data["target"]


C:\Users\Nirmal Choyal\AppData\Local\Temp\ipykernel_4160\836827921.py:2: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  data = kagglehub.load_dataset(


Download already complete (38114 bytes).


In [2]:
scaler = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

NameError: name 'StandardScaler' is not defined